In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pandas as pd
import numpy as np
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image

# Define and compile your model
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

# Load the CSV files
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Extract features (pixels) and labels from the training data
x_train = train_data.drop(columns=["label"]).values
y_train = train_data["label"].values

# Extract features (pixels) from the test data
x_test = test_data.values

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Normalize pixel values
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Convert labels to one-hot encoded format
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)

# Train your model
batch_size = 128
epochs = 3
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

# Save the trained model
model.save('pre_trained_model')
print("Saving the model as Pre_trained_model")

# Load the trained model
model = keras.models.load_model('mnist.h5')

def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert('L')
    img = np.array(img)
    # Reshape to support our model input and normalize
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    # Predict the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognise", command=self.classify_handwriting)   
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        a, b, c, d = rect
        rect = (a + 4, b + 4, c - 4, d - 4)
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')
       
app = App()
mainloop()


Epoch 1/3
329/329 [==============================] - 11s 32ms/step - loss: 2.3082 - accuracy: 0.1017
Epoch 2/3
329/329 [==============================] - 10s 30ms/step - loss: 2.2976 - accuracy: 0.1174
Epoch 3/3
329/329 [==============================] - 10s 30ms/step - loss: 2.2892 - accuracy: 0.1317


INFO:tensorflow:Assets written to: pre_trained_model\assets


INFO:tensorflow:Assets written to: pre_trained_model\assets


Saving the model as Pre_trained_model
1/1 [==============================] - 0s 14ms/step


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Extract features (pixels) and labels from the training data
x_train = train_data.drop(columns=["label"]).values
y_train = train_data["label"].values

# Extract features (pixels) from the test data
x_test = test_data.values

# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Reshape the data for CNN input
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Define the CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=128, verbose=2)

# Make predictions on the test data
predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)

# Create a DataFrame for submission
submission = pd.DataFrame({'ImageId': sample_submission['ImageId'], 'Label': predicted_labels})

# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)

# Calculate accuracy on the validation set
val_predictions = model.predict(x_val)
val_predicted_labels = np.argmax(val_predictions, axis=1)
val_accuracy = accuracy_score(y_val, val_predicted_labels)
print(f'Validation accuracy: {val_accuracy:.4f}')


Epoch 1/10
263/263 - 7s - loss: 0.2976 - accuracy: 0.9165 - val_loss: 0.1258 - val_accuracy: 0.9640 - 7s/epoch - 25ms/step
Epoch 2/10
263/263 - 6s - loss: 0.0911 - accuracy: 0.9743 - val_loss: 0.0892 - val_accuracy: 0.9720 - 6s/epoch - 24ms/step
Epoch 3/10
263/263 - 6s - loss: 0.0614 - accuracy: 0.9817 - val_loss: 0.0808 - val_accuracy: 0.9751 - 6s/epoch - 22ms/step
Epoch 4/10
263/263 - 6s - loss: 0.0447 - accuracy: 0.9865 - val_loss: 0.0639 - val_accuracy: 0.9787 - 6s/epoch - 21ms/step
Epoch 5/10
263/263 - 6s - loss: 0.0332 - accuracy: 0.9898 - val_loss: 0.0597 - val_accuracy: 0.9812 - 6s/epoch - 22ms/step
Epoch 6/10
263/263 - 6s - loss: 0.0270 - accuracy: 0.9920 - val_loss: 0.0592 - val_accuracy: 0.9830 - 6s/epoch - 21ms/step
Epoch 7/10
263/263 - 6s - loss: 0.0201 - accuracy: 0.9944 - val_loss: 0.0582 - val_accuracy: 0.9833 - 6s/epoch - 21ms/step
Epoch 8/10
263/263 - 6s - loss: 0.0147 - accuracy: 0.9959 - val_loss: 0.0556 - val_accuracy: 0.9833 - 6s/epoch - 22ms/step
Epoch 9/10
263/2

In [13]:
import tkinter as tk
import numpy as np
from PIL import ImageGrab, Image
import tensorflow as tf
from tensorflow import keras

# Load the trained model
model = keras.models.load_model('pre_trained_model')

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognize", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=tk.W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        self.label.configure(text="Draw..")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  
        rect = self.canvas.winfo_pointerxy() + (4, 4)  
        x1, y1, x2, y2 = self.canvas.bbox("all")
        rect = (x1, y1, x2, y2)
        im = ImageGrab.grab(rect)

        res = predict_digit(im)
        print(f"Raw Prediction Scores: {res}")  # Print the raw prediction scores
        digit = np.argmax(res)
        self.label.configure(text=f"Predicted Digit: {digit}")


    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

def predict_digit(img):
    img = img.resize((28, 28))
    img = img.convert('L')
    img = np.array(img)
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    res = model.predict([img])[0]
    return np.argmax(res)

app = App()
app.mainloop()


1/1 [==============================] - 0s 50ms/step
Raw Prediction Scores: 0
